# Homework 1

## Введение в OpenAI Gym

 <a href=https://gym.openai.com>OpenAI Gym</a> это набор инструментов для разработки и сравнения алгоритмов обучения с подкреплением.

OpenAI Gym предоставляет простой и универсальный API ко многим средам с разными свойствами, как простым так и сложным:
* Классические задачи управления и игрушечные примеры, которые можно найти в учебниках и на которых демонстрируется работа алгоритмов обучения с подкреплением (в основном они будут использоваться в этом курсе)
* Игры Atari (оказали огромное влияние на достижения в обучении с подкреплением в последние годы)
* 2D и 3D среды для контроля роботов в симуляции (используют проприетарный движок <a href=http://www.mujoco.org/>MuJoCo</a>)

Запустим агента со случайными действиями в среде <a href=https://gym.openai.com/envs/CartPole-v0>CartPole-v0</a>. В этой среде агент должен удержать шест, который закреплен на тележке, в вертикальном положении. Агент может прикладывать силу *+1* или *-1* к тележке в попытке удержать шест. Эпизод игры заканчивается, если шест отклоняется от вертикального положения больше, чем на *15* градусов или тележка сдвигается больше, чем на *2.4* единицы расстояния от центра. За каждый шаг времени, когда шест находится в вертикальном положении, агент получает награду в размере *+1* очко. Задаче считается "решенной" при получении агентом средней за 100 попыток награды *195* очков.

#### Для работы нам потребуется установить gym и numpy. Gym следует установить командой pip install gym=0.9.2

In [ ]:
# Импортируем необходимые библиотеки
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

In [ ]:
env = gym.make('CartPole-v0') # создаем среду CartPole-v0

In [ ]:
env.reset()
for _ in range(1000):
    env.render()
    observation, reward, done, info = env.step(env.action_space.sample()) # агент выбирает случайные действия
    if done:
        env.reset()

In [ ]:
env.close() # выключим визуализацию

Вспомним элементы проблемы обучения с подкреплением <img src="./scheme.png" width=500>

OpenAI Gym предоставляет такой же интерфейс взаимодействия со средой:
* Среда в ответ на действие агента предоставляет *bservation (object)*- специфичный для конкретной среды объект, который предствляет наблюдения агента. Например, пиксели камеры, значения углов и скоростей сочленений робота или позиции агента и других объектов в среде.
* *reward (float)* - значение награды, полученной агентом в резултате совершенного действия
* *done (boolean)* - флаг обозначающий окончание эпизода. Например, эпизод заканчивается, когда шест слишком сильно отклонился или агент попал в прорубь в среде FrozenLake
* *info (dict)* - словарь, содержащий диагностическую информацию, которую можно использовать для отладки, но не для обучения агента. Обычно мы присваеваем значение *info* переменной по-умолчению *_*

Теперь, когда мы познакомились с API OpenAI Gym, посмотрим, сколько очков награды сможет в среднем получить за 100 эпизодов агент, выбирающий случайные действия.

In [ ]:
total_reward = []
env = gym.make('CartPole-v0')
for episode in range(100):
    episode_reward = 0
    observation = env.reset()
    for t in range(100):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        episode_reward += reward
        if done:
            print("Episode {} finished after {} timesteps".format(episode+1, t+1))
            break
    total_reward.append(episode_reward)

In [ ]:
env.close()

In [ ]:
print(np.mean(total_reward))

Наш "cлучайный" агент получает в среднем 20 очков за 100 эпизодов. Не очень впечатляет.

В предыдущием эксперименте агент выбирал случайное действие. Важными объектами в OpenAI Gym являются пространства состояний и действий.

In [ ]:
env = gym.make('CartPole-v0')

In [ ]:
print(env.action_space.__doc__)

In [ ]:
print(env.action_space)

In [ ]:
print(env.action_space.n)

In [ ]:
print(env.observation_space.__doc__)

In [ ]:
print(env.observation_space)

In [ ]:
print(env.observation_space.shape)

In [ ]:
env.observation_space.high

In [ ]:
env.observation_space.low

## Value Iteration

На лекции мы рассмотрели, как мы можем выучить оптимальную политику, используя алгоритм Value Iteration, если нам известна динамика среды, а также если пространства состояний и действий не большие и дискретные.

Попробуем выучить оптимальную политику в среде <a href=https://gym.openai.com/envs/FrozenLake-v0>FrozenLake-v0</a>. Это простая среда с маленькими пространствами состояний и действий, а также с известной динамикой

Созданим среду и выведем её описание

In [ ]:
env = gym.make('FrozenLake-v0')

In [ ]:
print(env.env.__doc__)

Как видно среда представляет собой поле 4 на 4, по которому нужно добраться от начала (клетка *S*) до цели (клетка *G*). При этом среда является недетерменированный - с определенной вероятностью при совершения действия агент подскользнется и попадет не в ту клетку, в которую направлялся. Клетка *H* обозначает прорубь. Игра закначивается, когда агент попадает в клетку *G* или в клету *H*. Если агент проваливается в прорубь, то он получает награду *0*, если достигает клетки цели - *1*. 

Посмотрим, сколько в среднем очков награды за 100 эпизодов получит наш агент, если будет выполнять случайные действия.

In [ ]:
env.seed(0); from gym.spaces import prng; prng.seed(10) # установим сид для воспроизводимости результатов эксперимента

In [ ]:
total_reward = []
for episode in range(100):
    episode_reward = 0
    observation = env.reset()
    for t in range(100):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, _ = env.step(action)
        episode_reward += reward
        if done:
            print("Episode {} finished after {} timesteps".format(episode+1, t+1))
            break
    total_reward.append(episode_reward)

In [ ]:
print(np.mean(total_reward))

Как видим, только в 3 эпизодах из 100 агену удалось добраться до цели.

In [ ]:
env.reset()
for _ in range(100):
    env.render()
    action = env.action_space.sample() # take a random action
    observation, reward, done, _ = env.step(action)
    if done:
        break

Из среды OpenAI Gym мы можем получить элементы MDP (Markov Decision Process)

В env.env.P хранится двухуровневый словарь, в котором первый ключ является состояние, а второй - действием.
Клетки ассоциированыс индексами [0, 1, 2, ..., 15] слева направо и сверху вниз.

In [ ]:
print(np.arange(16).reshape(4,4))

Индексы действией [0, 1, 2, 3] соответствуют движению на Запад, Юг, Восток и Север.
env.env.P[state][action] возвращает лист кортежей (probability, nextstate, reward). Например, состояние 0 - это начальное состояние и информация о веротностях перехода для s=0 и a=0 содержит:

In [ ]:
env.env.P[0][0]

Другой пример - состояние 5 сооветсвует проруби, и все действия в данном состоянии приводят к тому же состоянию с вероятностью 1 и наградой 0

In [ ]:
for i in range(4):
    print("P[5][%i] =" % i, env.env.P[5][i])

Вспомним, что из себя представляет алгоритм Value Iteration <img src="./value_iteration.png" width="500">

Задание считается решенным, если агент доходит до цели в среднем в 70% эпизодов.

In [ ]:
n_states = env.env.nS
n_actions = env.env.nA
print("Number of states: {}".format(n_states))
print("Number of actions: {}".format(n_actions))

Напишем несклько вспомогательных функций.

Поскольку алгоритм Value Iteration возвращает нам оптимальную V-функцию, то нам необходимо извлекать из нее оптимальную политику (как указано в последней строке псевдокода алгоритма)

In [ ]:
def extract_policy(v, gamma = 1.0):
    policy = np.zeros(n_states)
    for state in range(n_states):
        q = np.zeros(n_actions)
        for action in range(n_actions):
            for next_sr in env.env.P[state][action]:
                probability, next_state, reward, _ = next_sr
                q[action] += (probability*(reward + gamma*v[next_state]))
        policy[state] = np.argmax(q)
    return policy

Также напишем функцию для оценки нашей найденной политики.

In [ ]:
def evaluate_policy(env, policy, gamma=1.0, n=100):
    total_reward = []
    for episode in range(n):
        episode_reward = 0
        observation = env.reset()
        step = 0
        for _ in range(100):
            env.render()
            action = int(policy[observation])
            observation, reward, done, _ = env.step(action)
            episode_reward += gamma**step*reward
            step += 1
            if done:
                break
        total_reward.append(episode_reward)
    return np.mean(total_reward)

Нам остается написать основную функцию, которая вернет оптимальную V-функцию.

In [ ]:
def value_iteration(env, gamma=1.0, max_iterations = 100000):
    v = np.zeros(n_states)
    eps = 1e-20
    # Your code goes here
    return v

Теперь мы можем найти оптимальную V-функцию, извлечь из нее оптимальную политику и оцениь ее.

In [ ]:
optimal_v = value_iteration(env)
optimal_policy = extract_policy(optimal_v)
optimal_policy_score = evaluate_policy(env, optimal_policy, n=100)

In [ ]:
print(optimal_v.reshape(4,4))

In [ ]:
print(optimal_policy.reshape(4,4))

In [ ]:
optimal_policy_score

По сравнению со "случайным" агентом, который доходил до цели в 3 случаях из 100, наша новая политика позволяет добирться до цели в ~70% эпизодов.

## Policy Iteration

Вспомним, что из себя представляет алгоритм Policy Iteration <img src="policy_iteration.png" width="500">

Напишем необходимые вспомогательные функции.

Начнем с основного цикла алгоритма, который вернет нам оптимальную политику.

In [ ]:
def policy_iteration(env, gamma=1.0, max_iterations = 200000):
    policy = np.random.choice(n_actions, size=(n_states))  # initialize a random policy
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)
        new_policy = extract_policy(old_policy_v, gamma)
        if (np.all(policy == new_policy)):
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        policy = new_policy
    return policy

А также еще раз напишем функцию для оценки найденной политики.

In [ ]:
def evaluate_policy(env, policy, gamma=1.0, n=100):
    total_reward = []
    for episode in range(n):
        episode_reward = 0
        observation = env.reset()
        step = 0
        for _ in range(100):
            env.render()
            action = int(policy[observation])
            observation, reward, done, _ = env.step(action)
            episode_reward += gamma**step*reward
            step += 1
            if done:
                break
        total_reward.append(episode_reward)
    return np.mean(total_reward)

Остается написать 2 функции, которые используются в основном цикле алгоритма Policy Iteration согласно псевдокоду.

In [ ]:
def compute_policy_v(env, policy, gamma=1.0):
    v = np.zeros(n_states)
    eps = 1e-10
    # Your code goes here
    return v

In [ ]:
def extract_policy(v, gamma=1.0):
    policy = np.zeros(n_states)
    # Your code goes here
    return policy

Теперь мы также можем найти оптимальную V-функцию, извлечь из нее оптимальную политику и оцениь ее.

In [ ]:
optimal_policy = policy_iteration(env)
optimal_policy_score = evaluate_policy(env, optimal_policy)

In [ ]:
print(optimal_policy.reshape(4,4))

In [ ]:
print(optimal_policy_score)

## Дополнительное задание

Сравнить поведение случайного агента с обученным вышеприведенными методами агентом в средах из OpenAI Gym с известной динамикой <a href="https://gym.openai.com/envs/Taxi-v2/">Taxi-v2</a> и CliffWalking. Посмотрите как ведет себя агент при использовании разных значений фактора дисконтирования. 

Ко всем средам OpenAi Gym можно получить доступ не только через *gym.make* но и через обычный импорт модулей. Среда CliffWalking ипортируется именно так. 

In [ ]:
from gym.envs import toy_text

In [ ]:
env = toy_text.CliffWalkingEnv()

In [ ]:
print(env.__doc__)